In [1]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = "/home/data/train.json"
with open(train_path, 'r') as f:
    train_data = json.load(f)

print(f"Number of training samples: {len(train_data)}")
print(f"Sample keys in a training record: {list(train_data[0].keys())}")
print("\nSample record:")
print(json.dumps(train_data[0], indent=2))

Number of training samples: 2878
Sample keys in a training record: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_request

In [ ]:
# Analyze target distribution
import matplotlib.pyplot as plt
import seaborn as sns

# Convert to DataFrame for easier analysis
df = pd.DataFrame(train_data)

# Target distribution
target_counts = df['requester_received_pizza'].value_counts()
print("Target Distribution:")
print(target_counts)
print(f"\nClass balance: {target_counts[True]} positive ({target_counts[True]/len(df)*100:.1f}%) vs {target_counts[False]} negative ({target_counts[False]/len(df)*100:.1f}%)")

# Plot target distribution
plt.figure(figsize=(8, 6))
sns.barplot(x=target_counts.index, y=target_counts.values)
plt.title('Distribution of Pizza Requests (Success vs Failure)')
plt.xlabel('Received Pizza')
plt.ylabel('Count')
plt.xticks([0, 1], ['No', 'Yes'])
plt.show()